In [ ]:
!pip install --upgrade pip
!pip install torch transformers soundfile
!pip install git+https://github.com/huggingface/parler-tts.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB

In [ ]:
!pip install TTS

In [ ]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 158.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [streamlit]


In [ ]:
%%writefile app.py
import os
os.environ["COQUI_TOS_AGREED"] = "1"
import streamlit as st
from TTS.api import TTS
import torch
from torch.serialization import add_safe_globals
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.config.shared_configs import BaseDatasetConfig

add_safe_globals([XttsAudioConfig, XttsArgs, XttsConfig, BaseDatasetConfig])


@st.cache_resource
def load_tts():
    return TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=torch.cuda.is_available())

tts_model = load_tts()

st.set_page_config(page_title="Ai Voice Cloner", page_icon="🧠")
st.title("AI Voice Cloner")

text_input = st.text_input("Enter Your Input Text")
options = {
    "Pirate": "/content/pirate.mp3",
    "Soldier": "/content/soldier.wav",
    "Young": "/content/this.wav",
    "Old":"/content/yeah.mp3"
}

choices = ["None"] + list(options.keys())
choice = st.selectbox("Select Voice", choices)
if choice != "None":
    st.write("You chose:", options[choice])
voice_file=st.file_uploader(
    "Upload Your Audio file",type=['mp3','wav','ogg'],help="Upload an audio for voice cloner"
)

submit=st.button('Submit')
if submit:
    if not text_input:
        st.error("❌ Please enter some text.")
    elif voice_file and choice != "None":
        st.error("❌ Please choose either a predefined voice OR upload a file, not both.")
    elif not voice_file and choice == "None":
        st.error("❌ Please select a predefined voice or upload your own audio file.")
    else:
        if voice_file:
            speaker_wav_path = "uploaded_voice.wav"
            with open(speaker_wav_path, "wb") as f:
                f.write(voice_file.read())
        else:
            speaker_wav_path = options[choice]

        output_path = "output.wav"

        tts_model.tts_to_file(
            text=text_input,
            speaker_wav=speaker_wav_path,
            file_path=output_path,
            language="en"
        )

        st.success("✅ Voice generated successfully!")

        if os.path.exists(output_path):
            with open(output_path, "rb") as audio_file:
                audio_bytes = audio_file.read()
                st.audio(audio_bytes, format='audio/wav')

Writing app.py


In [ ]:
!pip install git+https://github.com/coqui-ai/TTS.git

  Cloning https://github.com/coqui-ai/TTS.git to /tmp/pip-req-build-zmiplldy
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS.git /tmp/pip-req-build-zmiplldy
  Resolved https://github.com/coqui-ai/TTS.git to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TTS: filename=tts-0.22.0-cp311-cp311-linux_x86_64.whl size=904011 sha256=f513644537d49447d4c045d1b850c99e9411e991fdaefa8ce7ef7b7028ceb845
  Stored in directory: /tmp/pip-ephem-wheel-cache-81a4_fg7/wheels/4c/ea/e8/03164d56abc0fa08ae174fb00841a3e7e87327f2d3acc9273b
Successfully built TTS
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [TTS]


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2oZfkG35468OLNA2Zegmiy4XJ5u_41qzEBTC76rZUZkSpLSiz")

ngrok.kill()

public_url = ngrok.connect(addr=8501)
print(f"🌐 Streamlit app URL: {public_url}")
!pip install -q TTS streamlit torchaudio

# Run Streamlit
!streamlit run app.py &>/content/logs.txt &


🌐 Streamlit app URL: NgrokTunnel: "https://f36aadab7950.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from TTS.api import TTS
tts_model = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL TTS.tts.configs.xtts_config.XttsConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([XttsConfig])` or the `torch.serialization.safe_globals([XttsConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by 

In [ ]:
tts_model.tts_to_file(
    text="This is a test of the XTTS voice model.",
    speaker_wav="path/to/your/voice_sample.wav",
    language="en",
    file_path="out.wav"
)


In [ ]:
chunk = "Hello, my name is Prashant"
speaker_wav_path = "/content/pirate.mp3"  
output_path = "output.wav"

tts_model.tts_to_file(
    text=chunk,
    speaker_wav=speaker_wav_path, 
    file_path=output_path,
    language="en"  
)


 > Text splitted to sentences.
['Hello, my name is Prashant']
 > Processing time: 9.093440294265747
 > Real-time factor: 3.276313047198688


'output.wav'